In [3]:
from dash import Dash, html, dcc
from dash_table import DataTable
import plotly.express as px
import pandas as pd
import pandas as pd
import os
import redshift_connector

In [104]:
import os
os.environ["host"] = 'default-workgroup.127806285885.us-east-1.redshift-serverless.amazonaws.com'
os.environ["database"] = 'dev'
os.environ["user"] = 'admin'
os.environ["password"] = 'SEJVGrdfhj529-.'

In [105]:
conn = redshift_connector.connect(
    host=os.environ['host'],
    port=5439,
    database=os.environ["database"],
    user=os.environ["user"],
    password=os.environ["password"]
 )
cursor = conn.cursor()

In [106]:
cursor.execute("select * from congress_data")

In [28]:
df: pd.DataFrame = cursor.fetch_dataframe()

In [29]:
df

,prefix,first_name,last_name,suffix,name,filling_date,document_id,year,district,source_ptr_link,transcribed_by,owner,transaction_date,ticker,description,transaction_type,amount,cap_gains_over_200
0,Hon.,Earl,Blumenauer,,Hon. Earl Blumenauer,2022-05-05,20020943,2022,OR03,https://disclosures-clerk.house.gov/public_dis...,Timothy Carambat,self,2022-04-22,Y,Alleghany Corporation,sale_full,"$1,001 - $15,000",False
1,Hon.,Earl,Blumenauer,,Hon. Earl Blumenauer,2022-05-05,20020943,2022,OR03,https://disclosures-clerk.house.gov/public_dis...,Timothy Carambat,self,2022-03-31,--,Toyota Motor Credi 3.25% Due 04/01/25,purchase,"$15,001 - $50,000",False
2,Hon.,Earl,Blumenauer,,Hon. Earl Blumenauer,2022-05-05,20020943,2022,OR03,https://disclosures-clerk.house.gov/public_dis...,Timothy Carambat,self,2022-04-20,--,US Treasury NT 2.375% 04/26,purchase,"$15,001 - $50,000",False
3,Hon.,Earl,Blumenauer,,Hon. Earl Blumenauer,2022-05-05,20020943,2022,OR03,https://disclosures-clerk.house.gov/public_dis...,Timothy Carambat,self,2022-04-27,VOD,Vodafone Group plc - ADR shares each represent...,sale_partial,"$1,001 - $15,000",False
4,Hon.,William R.,Keating,,Hon. William R. Keating,2022-05-05,20020915,2022,MA09,https://disclosures-clerk.house.gov/public_dis...,Timothy Carambat,,2022-03-29,AMH,American Homes 4 Rent Common Shares of Benefic...,purchase,"$1,001 - $15,000",False
5,Hon.,William R.,Keating,,Hon. William R. Keating,2022-05-05,20020915,2022,MA09,https://disclosures-clerk.house.gov/public_dis...,Timothy Carambat,,2022-03-29,CCI,Crown Castle International Corp,sale_full,"$1,001 - $15,000",False
6,Hon.,William R.,Keating,,Hon. William R. Keating,2022-05-05,20020915,2022,MA09,https://disclosures-clerk.house.gov/public_dis...,Timothy Carambat,,2022-03-29,MDT,Medtronic plc Ordinary Shares,sale_full,"$1,001 - $15,000",False
7,Hon.,William R.,Keating,,Hon. William R. Keating,2022-05-05,20020915,2022,MA09,https://disclosures-clerk.house.gov/public_dis...,Timothy Carambat,,2022-03-29,VRTX,Vertex Pharmaceuticals Incorporated,purchase,"$1,001 - $15,000",False
8,Hon.,Harold Dallas,Rogers,,Hon. Harold Dallas Rogers,2022-05-05,8218698,2022,KY05,https://disclosures-clerk.house.gov/public_dis...,Timothy Carambat,,2022-02-01,--,Redeemed KY Infra Rev Waste & Drink Wtr 5.0 du...,sale_full,"$15,001 - $50,000",False


In [100]:
app = Dash(__name__)

app.layout = html.Div(children=[
    html.H1(children='Analytics report'),

    html.Div(children='''
        This is report created in dash for congress data analytics.
    '''),

    html.Div(children=[
        dcc.Graph(
        id='example-graph',
        figure=px.bar(df.groupby('amount').size().reset_index(name='no_transactions'), x="amount", y="no_transactions", barmode="group")
    ),
    ], style={'width': '50%', 'display': 'inline-block'}),
    
    html.Div(children=[
    dcc.Graph(
        id='example-graph2',
        figure=px.line(df.groupby('transaction_date').size().reset_index(name='no_transactions'), x="transaction_date", y="no_transactions",  markers=True,)
    ),
    ], style={'width': '50%', 'display': 'inline-block'}),
    
    html.Div(children=[
            dcc.Graph(
                id='pie-chart',
                figure=px.pie(df.groupby('transaction_type').size().reset_index(name='no_transactions'), values='no_transactions', names='transaction_type')
            ),
            # Add more components on the right side if needed
        ], style={'width': '50%', 'display': 'inline-block', 'vertical-align': 'middle'}),
    
    # Table
    html.Div([
        DataTable(
            id='table',
            columns=[{'name': col, 'id': col} for col in df.groupby(['name', 'cap_gains_over_200']).size().reset_index(name='no_transactions').columns],
            data=df.groupby(['name', 'cap_gains_over_200']).size().reset_index(name='no_transactions').to_dict('records')
        )
    ], style={'width': '40%', 'display': 'inline-block'})
    
])

if __name__ == '__main__':
    app.run(debug=True)